In [1]:
# Cell 1: Setup Environment and Install Packages
import os
import warnings
warnings.filterwarnings('ignore')

# Create directories
os.makedirs('medical_rag_bot', exist_ok=True)
os.makedirs('medical_rag_bot/docs', exist_ok=True)
os.chdir('/content/medical_rag_bot')

# Install required packages
!pip install -q google-generativeai langchain langchain-google-genai chromadb PyMuPDF langchain-community sentence-transformers faiss-cpu

print("✅ Medical RAG Bot Environment Setup Complete!")
print("📁 Directories created: medical_rag_bot/, medical_rag_bot/docs/")
print("📦 All packages installed successfully!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.8 MB/s eta 0:00:0

In [2]:
# CELL 2: API Configuration & Imports
# =============================================================================
from google.colab import userdata
import google.generativeai as genai
from langchain.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
import time
import asyncio
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# Get API key from Colab secrets
try:
    API_KEY = userdata.get('MY_GEMINI_KEY')
    os.environ["GOOGLE_API_KEY"] = API_KEY
    genai.configure(api_key=API_KEY)
    print("✅ Medical Bot API configured successfully!")
    print("🔐 Gemini API key loaded from secrets")
except Exception as e:
    print("❌ Error configuring API key. Please check your GEMINI_API_KEY in Colab secrets")
    print(f"Error: {e}")


✅ Medical Bot API configured successfully!
🔐 Gemini API key loaded from secrets


In [3]:
# CELL 3: Document Upload
# =============================================================================
from google.colab import files
import shutil

print("📋 Medical Document Upload")
print("💡 Please upload your medical PDF or any medical documentation")
print("⚡ Supported formats: PDF files only")
print("-" * 50)

uploaded = files.upload()

if uploaded:
    for filename in uploaded.keys():
        if filename.endswith('.pdf'):
            shutil.move(filename, f'docs/{filename}')
            print(f"✅ Medical document uploaded: {filename}")
        else:
            print(f"⚠️  Skipping non-PDF file: {filename}")

    pdf_count = len([f for f in os.listdir('docs') if f.endswith('.pdf')])
    print(f"\n🏥 {pdf_count} medical document(s) ready for processing")
else:
    print("ℹ️  No files uploaded. You can run this cell again to upload documents.")

📋 Medical Document Upload
💡 Please upload your medical PDF or any medical documentation
⚡ Supported formats: PDF files only
--------------------------------------------------


Saving Medical_book.pdf to Medical_book.pdf
✅ Medical document uploaded: Medical_book.pdf

🏥 1 medical document(s) ready for processing


In [4]:
# CELL 4: Document Processing
# =============================================================================
print("📖 Processing medical documents...")
print("⏳ This may take a few minutes for large documents (637+ pages)")
print("-" * 50)

documents = []
start_time = time.time()

# Load all PDF documents
for filename in os.listdir("docs"):
    if filename.endswith(".pdf"):
        print(f"📄 Loading: {filename}")
        loader = PyMuPDFLoader(f"docs/{filename}")
        docs = loader.load()
        documents.extend(docs)
        print(f"   ➤ {len(docs)} pages loaded")

if not documents:
    print("❌ No documents found. Please upload PDF files first.")
else:
    # Split documents into chunks optimized for medical content
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,  # Larger chunks for medical context
        chunk_overlap=300,  # More overlap for better continuity
        separators=["\n\n", "\n", ". ", ", ", " ", ""]
    )

    print(f"\n🔧 Splitting documents into chunks...")
    chunks = text_splitter.split_documents(documents)

    processing_time = time.time() - start_time
    print(f"\n✅ Document processing complete!")
    print(f"📊 Statistics:")
    print(f"   • Total pages: {len(documents)}")
    print(f"   • Text chunks: {len(chunks)}")
    print(f"   • Processing time: {processing_time:.1f} seconds")
    print(f"   • Average chunk size: ~{1500} characters")


📖 Processing medical documents...
⏳ This may take a few minutes for large documents (637+ pages)
--------------------------------------------------
📄 Loading: Medical_book.pdf
   ➤ 637 pages loaded

🔧 Splitting documents into chunks...

✅ Document processing complete!
📊 Statistics:
   • Total pages: 637
   • Text chunks: 2335
   • Processing time: 7.8 seconds
   • Average chunk size: ~1500 characters


In [5]:
# CELL 5: Vector Database Creation
# =============================================================================
print("🧬 Creating medical knowledge vector database...")
print("⏳ Generating embeddings for medical content...")
print("-" * 50)

start_time = time.time()

try:
    # Initialize embeddings model
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

    # Create vector database with persistence
    vectordb = Chroma.from_documents(
        chunks,
        embeddings,
        persist_directory='medical_db',
        collection_name="medical_knowledge"
    )

    # Set up retriever with medical-optimized settings
    retriever = vectordb.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5}
    )

    # Initialize LLM with medical-appropriate settings
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0.1,  # Lower temperature for medical accuracy
        max_tokens=1024
    )

    embedding_time = time.time() - start_time

    print(f"✅ Medical vector database created successfully!")
    print(f"📊 Database info:")
    print(f"   • Embeddings model: Google embedding-001")
    print(f"   • LLM model: Gemini 1.5 Flash")
    print(f"   • Database location: medical_db/")
    print(f"   • Processing time: {embedding_time:.1f} seconds")
    print(f"   • Retrieval strategy: Top-5 similarity search")

except Exception as e:
    print(f"❌ Error creating vector database: {e}")
    print("Please check your API key and internet connection.")

🧬 Creating medical knowledge vector database...
⏳ Generating embeddings for medical content...
--------------------------------------------------
✅ Medical vector database created successfully!
📊 Database info:
   • Embeddings model: Google embedding-001
   • LLM model: Gemini 1.5 Flash
   • Database location: medical_db/
   • Processing time: 35.0 seconds
   • Retrieval strategy: Top-5 similarity search


In [6]:
# CELL 6: Clean Improved Search Function
# =============================================================================
print("🔧 Loading improved search with clean output...")

def improved_search_with_rag(query):
    """
    Improved search with better query variations and semantic understanding
    """
    # IMPROVED search variations
    base_query = query.lower()
    search_variations = []

    # Original query
    search_variations.append(query)

    # Clean query (remove question words)
    clean_query = query
    for remove_word in ["what are", "what is", "tell me about", "explain", "describe", "how is", "how are"]:
        clean_query = clean_query.replace(remove_word, "").strip()
    if clean_query != query:
        search_variations.append(clean_query)

    # Keyword extraction (last 2-3 meaningful words)
    words = query.split()
    if len(words) >= 2:
        search_variations.append(" ".join(words[-2:]))  # Last 2 words
    if len(words) >= 3:
        search_variations.append(" ".join(words[-3:]))  # Last 3 words

    # Semantic expansion based on question type
    if "symptom" in query.lower():
        condition_words = [w for w in words if w.lower() not in ["what", "are", "is", "the", "of", "symptoms", "symptom"]]
        if condition_words:
            condition = " ".join(condition_words)
            search_variations.extend([
                f"{condition} symptoms",
                f"symptoms {condition}",
                f"{condition} signs",
                f"{condition} manifestations",
                f"{condition} effects",
                f"{condition} complications"
            ])
    elif "treat" in query.lower():
        condition_words = [w for w in words if w.lower() not in ["what", "is", "how", "treat", "treatment", "treated"]]
        if condition_words:
            condition = " ".join(condition_words)
            search_variations.extend([
                f"{condition} treatment",
                f"{condition} therapy",
                f"{condition} management",
                f"treating {condition}"
            ])
    elif "cause" in query.lower():
        condition_words = [w for w in words if w.lower() not in ["what", "causes", "cause", "caused"]]
        if condition_words:
            condition = " ".join(condition_words)
            search_variations.extend([
                f"{condition} causes",
                f"{condition} etiology",
                f"{condition} risk factors"
            ])

    # Remove duplicates while preserving order
    unique_variations = []
    for var in search_variations:
        if var.strip() and var not in unique_variations:
            unique_variations.append(var)

    # Search with all variations and collect results
    all_results = []
    seen_content = set()

    for search_term in unique_variations:
        docs = vectordb.similarity_search(search_term, k=8)  # Get more results per variation

        for doc in docs:
            content_hash = hash(doc.page_content[:100])
            if content_hash not in seen_content:
                seen_content.add(content_hash)
                all_results.append(doc)

    # Take top 10 unique results for better coverage
    final_chunks = all_results[:10]

    if not final_chunks:
        return "❌ I don't have enough information in the medical encyclopedia to answer this question accurately. Please consult a healthcare professional.", []

    # Create context
    context_text = "\n\n".join([
        f"[ENCYCLOPEDIA SECTION {i+1}]:\n{doc.page_content}"
        for i, doc in enumerate(final_chunks)
    ])

    # ENHANCED RAG prompt with semantic understanding
    rag_prompt = f"""You are analyzing content from the Gale Encyclopedia of Medicine to answer a medical question.

ENCYCLOPEDIA CONTENT:
{context_text}

QUESTION: {query}

CRITICAL INSTRUCTIONS:
1. Focus on SEMANTIC RELEVANCE - look for information that answers the MEANING of the question, not just exact phrase matches
2. If asked about "symptoms," look for any descriptions of: signs, symptoms, manifestations, effects, complications, clinical presentations, or how the condition affects patients
3. If asked about "treatment," look for: therapy, management, medication, procedures, interventions, or therapeutic approaches
4. If asked about "causes," look for: etiology, risk factors, pathophysiology, or contributing factors
5. Extract ALL relevant information that addresses the question's intent, even if worded differently
6. Look across ALL encyclopedia sections for complementary information
7. Synthesize information from multiple sections when they relate to the same topic
8. Quote exact phrases when describing specific medical findings or lists
9. If you find semantically relevant information, provide a comprehensive answer
10. If the encyclopedia content does NOT contain information that meaningfully addresses the question, respond with: "INSUFFICIENT_ENCYCLOPEDIA_INFO"
11. Include appropriate medical disclaimers

SEMANTIC MATCHING EXAMPLES:
- "What are diabetes symptoms?" → Look for descriptions of how diabetes affects patients, clinical signs, manifestations
- "How is cancer treated?" → Look for therapy approaches, medications, procedures, management strategies
- "What causes heart disease?" → Look for risk factors, etiology, pathophysiology, contributing conditions

Pay attention to the MEANING and INTENT of the question, not just keyword matching.

ENCYCLOPEDIA-BASED ANSWER:"""

    try:
        response = llm.invoke(rag_prompt)
        content = response.content if hasattr(response, "content") else str(response)

        if "INSUFFICIENT_ENCYCLOPEDIA_INFO" in content.upper():
            return "❌ I don't have enough information in the medical encyclopedia to answer this question accurately. Please consult a healthcare professional.", final_chunks

        return content, final_chunks

    except Exception as e:
        return f"❌ Error analyzing encyclopedia content: {e}", final_chunks

print("✅ Clean search function loaded!")
print("🎯 Semantic understanding with minimal output noise")

🔧 Loading improved search with clean output...
✅ Clean search function loaded!
🎯 Semantic understanding with minimal output noise


In [7]:
# CELL 7: Clean LLM Fallback System
# =============================================================================
print("🔧 Setting up LLM Fallback System...")

def ask_llm_only(query):
    """
    Try LLM first - if uncertain, return None to trigger RAG search
    """
    prompt = f"""You are a medical AI assistant. Answer the following medical question ONLY if you are completely confident and certain about the answer.

STRICT INSTRUCTIONS:
- If you are even slightly uncertain, unsure, or lack complete confidence, respond with exactly: "MEDICAL_UNCERTAIN"
- Only provide answers for well-established medical knowledge you are absolutely certain about
- Do not guess, infer, or provide partial answers
- Do not provide specific medical advice, diagnoses, or treatment recommendations
- Always include appropriate medical disclaimers when you do answer

Question: {query}

Answer only if completely certain, otherwise respond "MEDICAL_UNCERTAIN":"""

    try:
        response = llm.invoke(prompt)
        content = response.content if hasattr(response, "content") else str(response)

        # Check if LLM indicated uncertainty
        if any(phrase in content.upper() for phrase in ["MEDICAL_UNCERTAIN", "UNCERTAIN", "UNSURE", "DON'T KNOW", "NOT SURE", "I DON'T", "CANNOT BE CERTAIN"]):
            return None

        return content
    except Exception as e:
        print(f"❌ LLM Error: {e}")
        return None

def ask_with_rag(query):
    """
    Use RAG when LLM is uncertain - search encyclopedia and analyze
    """
    print("   🔍 Searching encyclopedia...")

    # Multiple search strategies with more results
    search_variations = [
        query,
        query.replace("what are", "").replace("what is", "").replace("?", "").strip(),
        f"treatment {query.split()[-1]}" if any(word in query.upper() for word in ["AIDS", "HIV", "CANCER", "DIABETES"]) else query,
        " ".join(query.split()[-3:])  # Last 3 words
    ]

    all_chunks = []
    seen_content = set()

    for search_term in search_variations:
        docs = vectordb.similarity_search(search_term, k=10)

        for doc in docs:
            content_hash = hash(doc.page_content[:100])
            if content_hash not in seen_content:
                seen_content.add(content_hash)
                all_chunks.append(doc)

    # Keep top 6 chunks
    final_chunks = all_chunks[:6]

    if not final_chunks:
        return "❌ I don't have enough information in the medical encyclopedia to answer this question accurately. Please consult a healthcare professional.", []

    print(f"   📄 Analyzing {len(final_chunks)} sections...")

    # Create context from retrieved chunks
    context_text = "\n\n".join([
        f"[ENCYCLOPEDIA SECTION {i+1}]:\n{doc.page_content}"
        for i, doc in enumerate(final_chunks)
    ])

    # RAG prompt focused on encyclopedia content
    rag_prompt = f"""You are analyzing content from the Gale Encyclopedia of Medicine to answer a medical question.

ENCYCLOPEDIA CONTENT:
{context_text}

QUESTION: {query}

CRITICAL INSTRUCTIONS:
1. Focus on SEMANTIC RELEVANCE - look for information that answers the MEANING of the question, not just exact phrase matches
2. If asked about "symptoms," look for any descriptions of: signs, symptoms, manifestations, effects, complications, clinical presentations, or how the condition affects patients
3. If asked about "treatment," look for: therapy, management, medication, procedures, interventions, or therapeutic approaches
4. If asked about "causes," look for: etiology, risk factors, pathophysiology, or contributing factors
5. Extract ALL relevant information that addresses the question's intent, even if worded differently
6. Look across ALL encyclopedia sections for complementary information
7. Synthesize information from multiple sections when they relate to the same topic
8. Quote exact phrases when describing specific medical findings or lists
9. If you find semantically relevant information, provide a comprehensive answer
10. If the encyclopedia content does NOT contain information that meaningfully addresses the question, respond with: "INSUFFICIENT_ENCYCLOPEDIA_INFO"
11. Include appropriate medical disclaimers

ENCYCLOPEDIA-BASED ANSWER:"""

    try:
        response = llm.invoke(rag_prompt)
        content = response.content if hasattr(response, "content") else str(response)

        # Check if encyclopedia had insufficient info
        if "INSUFFICIENT_ENCYCLOPEDIA_INFO" in content.upper():
            return "❌ I don't have enough information in the medical encyclopedia to answer this question accurately. Please consult a healthcare professional.", final_chunks

        return content, final_chunks

    except Exception as e:
        return f"❌ Error analyzing encyclopedia content: {e}", final_chunks

def medical_fallback_response(query):
    """
    Clean version: Try LLM first, fallback to RAG if uncertain, admit ignorance if neither works
    """
    # Step 1: Try LLM direct knowledge first
    llm_response = ask_llm_only(query)

    if llm_response:
        # LLM was confident
        return f"[🧠 DIRECT] {llm_response}", "direct", [], False

    # Step 2: LLM was uncertain, try RAG
    rag_response, chunks = improved_search_with_rag(query)

    if rag_response and not rag_response.startswith("❌"):
        # RAG found good information
        return f"[📚 RAG] {rag_response}", "rag", chunks, True

    # Step 3: Neither LLM nor RAG could answer confidently
    return """[❌ UNKNOWN] I don't have enough reliable information to answer this question accurately.

This could be because:
• The question is outside my knowledge base
• The medical encyclopedia doesn't contain this specific information
• The topic requires specialized expertise

🏥 RECOMMENDATION: Please consult a qualified healthcare professional who can provide accurate, personalized medical information for your specific situation.

⚠️ It's always better to admit uncertainty than to provide potentially incorrect medical information.""", "unknown", [], False

def show_encyclopedia_sources(chunks, show_chunks=False):
    """
    Display encyclopedia sources cleanly
    """
    if not chunks:
        return

    if show_chunks:
        print(f"\n📋 Sources:")
        for i, chunk in enumerate(chunks):
            page_info = f"Page {chunk.metadata.get('page', 'Unknown')}"
            preview = chunk.page_content[:200] + "..."
            print(f"• {page_info}: {preview}")

print("✅ Clean LLM Fallback System loaded!")
print("🎨 Simplified processing messages")

🔧 Setting up LLM Fallback System...
✅ Clean LLM Fallback System loaded!
🎨 Simplified processing messages


In [10]:
# CELL 8: Simple Medical Chat Interface (Replace your current Cell 8 with this)
# =============================================================================

class MedicalChat:
    def __init__(self):
        self.show_sources = False
        self.running = True

    def start_chat(self):
        """Start the interactive medical chat"""
        self.print_header()
        self.chat_loop()

    def print_header(self):
        """Print the chatbot header"""
        print("🏥 MEDICAL LLM FALLBACK CHATBOT")
        print("="*60)
        print("🔄 Smart Response System:")
        print("   • 🧠 DIRECT: General medical knowledge")
        print("   • 📚 RAG: Encyclopedia-based research")
        print("   • ❌ UNKNOWN: Admits uncertainty")
        print("\n🛡️ Safety: Prevents misinformation • Admits limitations")
        print("⚠️  Educational purposes only - consult healthcare professionals")
        print("-" * 60)

        if not documents:
            print("⚠️  No medical documents loaded!")
            return False
        else:
            print(f"📚 Medical Encyclopedia: {len(documents)} pages loaded")

        print("\n💡 Example questions to try:")
        print("   🧠 'What is diabetes?' (likely DIRECT)")
        print("   📚 'What are the four considerations for AIDS treatment?' (likely RAG)")
        print("   ❌ 'What COVID variant was discovered yesterday?' (likely UNKNOWN)")
        print("\n💡 Commands: 'sources on/off', 'exit'")
        return True

    def chat_loop(self):
        """Main chat interaction loop"""
        while self.running:
            try:
                user_input = input("\n💬 Your question: ").strip()

                if not user_input:
                    continue

                self.handle_input(user_input)

            except KeyboardInterrupt:
                print("\n\n👋 Goodbye!")
                break
            except Exception as e:
                print(f"\n❌ Error: {e}")

    def handle_input(self, user_input):
        """Process user input and commands"""
        command = user_input.lower()

        # Handle commands
        if command in ['exit', 'quit', 'stop']:
            print("\n👋 Thank you for using the medical chatbot!")
            self.running = False
            return

        elif command == 'sources on':
            self.show_sources = True
            print("✅ Detailed sources enabled")
            return

        elif command == 'sources off':
            self.show_sources = False
            print("✅ Detailed sources disabled")
            return

        # Process medical question
        self.process_question(user_input)

    def process_question(self, question):
        """Process a medical question"""
        print(f"\n💬 {question}")

        # Get response using your existing function
        response, mode, chunks, has_sources = medical_fallback_response(question)

        # Display response
        self.display_response(response, chunks, has_sources)
        print("-" * 50)

    def display_response(self, response, chunks, has_sources):
        """Display the response based on type"""
        if "[🧠 DIRECT]" in response:
            clean_text = response.replace("[🧠 DIRECT]", "").strip()
            print(f"\n🧠 DIRECT: {clean_text}")

        elif "[📚 RAG]" in response:
            clean_text = response.replace("[📚 RAG]", "").strip()
            print(f"\n📚 RAG: {clean_text}")
            self.show_source_info(chunks, has_sources)

        elif "[❌ UNKNOWN]" in response:
            clean_text = response.replace("[❌ UNKNOWN]", "").strip()
            print(f"\n❌ UNKNOWN: {clean_text}")

    def show_source_info(self, chunks, has_sources):
        """Show source information if available"""
        if has_sources and chunks:
            if self.show_sources:
                print(f"\n📋 Detailed Sources:")
                for i, chunk in enumerate(chunks[:3]):
                    page_num = chunk.metadata.get('page', 'Unknown')
                    preview = chunk.page_content[:200] + "..."
                    print(f"• Page {page_num}: {preview}")
            else:
                page_list = [str(chunk.metadata.get('page', '?')) for chunk in chunks[:5]]
                print(f"\n📋 Sources: Encyclopedia pages {', '.join(page_list)}")

def ask_single_question(question, show_sources=False):
    """Ask a single question without starting interactive chat"""
    if not question.strip():
        print("❌ Please provide a question.")
        return

    print(f"💬 {question}")

    # Get response
    response, mode, chunks, has_sources = medical_fallback_response(question)

    # Create temporary chat instance to use display method
    chat = MedicalChat()
    chat.show_sources = show_sources
    chat.display_response(response, chunks, has_sources)
    print("-" * 50)

# Create the chat instance
medical_chat = MedicalChat()

# Show header immediately
medical_chat.print_header()

print("\n" + "="*50)
print("🚀 HOW TO USE:")
print("="*50)
print("Interactive Chat:")
print("   medical_chat.start_chat()")
print("\nSingle Questions:")
print("   ask_single_question('What is diabetes?')")
print("   ask_single_question('AIDS treatment', show_sources=True)")
print("="*50)

🏥 MEDICAL LLM FALLBACK CHATBOT
🔄 Smart Response System:
   • 🧠 DIRECT: General medical knowledge
   • 📚 RAG: Encyclopedia-based research
   • ❌ UNKNOWN: Admits uncertainty

🛡️ Safety: Prevents misinformation • Admits limitations
⚠️  Educational purposes only - consult healthcare professionals
------------------------------------------------------------
📚 Medical Encyclopedia: 637 pages loaded

💡 Example questions to try:
   🧠 'What is diabetes?' (likely DIRECT)
   📚 'What are the four considerations for AIDS treatment?' (likely RAG)
   ❌ 'What COVID variant was discovered yesterday?' (likely UNKNOWN)

💡 Commands: 'sources on/off', 'exit'

🚀 HOW TO USE:
Interactive Chat:
   medical_chat.start_chat()

Single Questions:
   ask_single_question('What is diabetes?')
   ask_single_question('AIDS treatment', show_sources=True)


In [ ]:
medical_chat.start_chat()

🏥 MEDICAL LLM FALLBACK CHATBOT
🔄 Smart Response System:
   • 🧠 DIRECT: General medical knowledge
   • 📚 RAG: Encyclopedia-based research
   • ❌ UNKNOWN: Admits uncertainty

🛡️ Safety: Prevents misinformation • Admits limitations
⚠️  Educational purposes only - consult healthcare professionals
------------------------------------------------------------
📚 Medical Encyclopedia: 637 pages loaded

💡 Example questions to try:
   🧠 'What is diabetes?' (likely DIRECT)
   📚 'What are the four considerations for AIDS treatment?' (likely RAG)
   ❌ 'What COVID variant was discovered yesterday?' (likely UNKNOWN)

💡 Commands: 'sources on/off', 'exit'

💬 Your question: what is diabetes

💬 what is diabetes

🧠 DIRECT: Diabetes is a group of metabolic disorders characterized by hyperglycemia resulting from defects in insulin secretion, insulin action, or both.  This leads to elevated levels of glucose in the blood.

**Disclaimer:** This information is for general knowledge and educational purposes only